<a href="https://colab.research.google.com/github/yukitiec/Research/blob/main/visionTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GPU setting

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

#Import Library

In [31]:
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense,SimpleRNN, LSTM,Dropout,Convolution2D,MaxPooling2D
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import glob 
import os 
import cv2
import matplotlib.pyplot as plt
import random #random.shuffle() : shuffle the data of 1D
from sklearn.utils import shuffle

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Load Image

In [32]:
## ///////// loading the data as DataFrame ////////////
#20220401
#file=pd.read_csv(filepath_or_buffer="/content/gdrive/My Drive/YAMAKAWA_LAB/技術補佐員/NN/dataset/train_dataset_2.csv")
#file_test = pd.read_csv(filepath_or_buffer= "/content/gdrive/My Drive/YAMAKAWA_LAB/技術補佐員/NN/dataset/test_dataset.csv")

#20220603
file_train = pd.read_csv(filepath_or_buffer = "/content/gdrive/My Drive/YAMAKAWA_LAB/技術補佐員/動画/20221020/dataset/train_vit.csv")
value_train = file_train.values
file_test = pd.read_csv(filepath_or_buffer = "/content/gdrive/My Drive/YAMAKAWA_LAB/技術補佐員/動画/20221020/dataset/test_vit.csv")
value_test = file_test.values

print(value_test[:5])
## ////////// テスト用とトレーニング用でデータをシャッフルする //////////////
#for 2D shuffle

value_train = shuffle(value_train,random_state=36) #random_state = integer ; fix randomness with shuffling
value_test = shuffle(value_test,random_state=42)
#print(value_train.shape)
#value_test = shuffle(value_test, random_state =42)
#print(value_test.shape)

## //////////// Show example /////////
#img=cv2.imread(value_train[100,1])
#gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#print(gray.shape)
#plt.imshow(gray)

## ////// making the dataset for train and test //////////

data_train = value_train[:,1:]
label_train =  value_train[:,0]


data_test = value_test[:,1:]
label_test = value_test[:,0]

count_test = 0
count_train = 0
for i in range(len(label_test)):
  if label_test[i] == 1:
    count_test += 1

for i in range(len(label_train)):
  if label_train[i] == 1:
    count_train += 1

print('spatter label : : train : {}, test : {}'.format(count_train,count_test))
#print(data_test)
print(len(label_train))

[[0
  '/content/gdrive/My Drive/YAMAKAWA_LAB/技術補佐員/動画/20221020/video/XIMEA_221020_23_trim/concat2/0039.jpg']
 [0
  '/content/gdrive/My Drive/YAMAKAWA_LAB/技術補佐員/動画/20221020/video/XIMEA_221020_23_trim/concat2/0042.jpg']
 [0
  '/content/gdrive/My Drive/YAMAKAWA_LAB/技術補佐員/動画/20221020/video/XIMEA_221020_23_trim/concat2/0049.jpg']
 [0
  '/content/gdrive/My Drive/YAMAKAWA_LAB/技術補佐員/動画/20221020/video/XIMEA_221020_23_trim/concat2/0043.jpg']
 [0
  '/content/gdrive/My Drive/YAMAKAWA_LAB/技術補佐員/動画/20221020/video/XIMEA_221020_23_trim/concat2/0036.jpg']]
spatter label : : train : 748, test : 104
1467


##Convert URL to Images

In [9]:
print(data_train.shape)

(1467, 1)


In [29]:
IMG_SIZE = 64*3
def url2image(data,label,img_size=IMG_SIZE):
  image_data = np.zeros((len(data),img_size,img_size))
  label_revised = []
  for i,[name] in enumerate(data):
    print((len(data),i))
    #print(name)
    img=cv2.imread(name)
    if img:
      gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      image_data[i] = gray
      label_revised.append(label[i])
  print("length of dataset is ",len(label_revised))
  imagedata = image_data[:len(label_revised)]

  return image_data,label_revised

In [30]:
image_train,label_train = url2image(data_train,label_train)
image_test,label_test = url2image(data_test,label_test)

(1467, 0)
(1467, 1)


ValueError: ignored


##Process 

In [ ]:


## split data into training data and test data 
image_train = image_train.reshape(len(image_train),10,64,64,1)
image_test = image_test.reshape(len(image_test),10,64,64,1)

#when calculating (dividing process) the type have to be float
image_train = image_train.astype("float32")
image_test = image_test.astype("float32")

#when training, it is convenient if the values are normalized
image_train= image_train/255
image_test = image_test/255

#for validation data
#num_sample = int(len(image_train)*0.8)
#image_train = image_train#[:num_sample]
#image_val = image_train#[num_sample:]
#abel_train = label_train[:num_sample]
#label_val = label_train[num_sample:]

#converting data to float32, especially float32
image_train =np.asarray(image_train).astype(np.float32)
label_train = np.asarray(label_train).astype(np.int32)
image_test =np.asarray(image_test).astype(np.float32)
label_test = np.asarray(label_test).astype(np.int32)


print("length of label test : ",len(label_test))

#label_train_tensorflow=keras.utils.to_categorical(label_train)  #keras.utils.to_categorical : can see how the result of the judge is like [0.4,0.6] : no-spatter: 40%, spatter : 60%
#label_test_tensorflow=keras.utils.to_categorical(label_test)
#print(image_train)
print(image_train.shape)
print(len(label_train))

length of label test :  460
(3755, 10, 64, 64, 1)
3755


In [ ]:
# convert label into array(2) [no-spatter, spatter]
def label_vec(label,dim):
  count=0
  lab = np.zeros((len(label),dim))  
  for i in range(len(label)):
    if label[i] == 1: #spatter
      lab[i,1] = 1
      count+=1
    else: #no spatter
      lab[i,0] = 1
  print('spatter label:{}'.format(count))
  return lab

lab_train = label_vec(label_train, 2)
lab_test = label_vec(label_test,2)

print("lab_train.shape ; " , lab_train.shape)
print("lab_test.shape : " , lab_test.shape)

spatter label:1966
spatter label:190
lab_train.shape ;  (3755, 2)
lab_test.shape :  (460, 2)


#Model Setting

##Input Layer

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [37]:
IN_CHANNEL = 1
EMB_DIM = 384
NUM_PATCH_ROW = 3
IMG_SIZE = 64*3

class VitInputLayer(nn.Module): 
    def __init__(self, in_channels:int=IN_CHANNEL, emb_dim:int=EMB_DIM, num_patch_row:int=NUM_PATCH_ROW, image_size:int=IMG_SIZE):
        """ 
        引数:
            in_channels: 入力画像のチャンネル数
            emb_dim: 埋め込み後のベクトルの長さ
            num_patch_row: 高さ方向のパッチの数。例は2x2であるため、2をデフォルト値とした 
            image_size: 入力画像の1辺の大きさ。入力画像の高さと幅は同じであると仮定
        """
        super(VitInputLayer, self).__init__() 
        self.in_channels=in_channels 
        self.emb_dim = emb_dim 
        self.num_patch_row = num_patch_row 
        self.image_size = image_size
        
        # パッチの数
        ## 例: 入力画像を2x2のパッチに分ける場合、num_patchは4 
        self.num_patch = self.num_patch_row**2

        # パッチの大きさ
        ## 例: 入力画像の1辺の大きさが32の場合、patch_sizeは16 
        self.patch_size = int(self.image_size // self.num_patch_row)

        # 入力画像のパッチへの分割 & パッチの埋め込みを一気に行う層 
        self.patch_emb_layer = nn.Conv2d(
            in_channels=self.in_channels, 
            out_channels=self.emb_dim, 
            kernel_size=self.patch_size, 
            stride=self.patch_size
        )

        # クラストークン 
        self.cls_token = nn.Parameter(
            torch.randn(1, 1, emb_dim) 
        )

        # 位置埋め込み
        ## クラストークンが先頭に結合されているため、
        ## 長さemb_dimの位置埋め込みベクトルを(パッチ数+1)個用意 
        self.pos_emb = nn.Parameter(
            torch.randn(1, self.num_patch+1, emb_dim) 
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """ 
        引数:
            x: 入力画像。形状は、(B, C, H, W)。[式(1)]
                B: バッチサイズ、C:チャンネル数、H:高さ、W:幅

        返り値:
            z_0: ViTへの入力。形状は、(B, N, D)。
                B:バッチサイズ、N:トークン数、D:埋め込みベクトルの長さ
        """
        # パッチの埋め込み & flatten [式(3)]
        ## パッチの埋め込み (B, C, H, W) -> (B, D, H/P, W/P) 
        ## ここで、Pはパッチ1辺の大きさ
        z_0 = self.patch_emb_layer(x)

        ## パッチのflatten (B, D, H/P, W/P) -> (B, D, Np) 
        ## ここで、Npはパッチの数(=H*W/Pˆ2)
        z_0 = z_0.flatten(2)

        ## 軸の入れ替え (B, D, Np) -> (B, Np, D) 
        z_0 = z_0.transpose(1, 2)

        # パッチの埋め込みの先頭にクラストークンを結合 [式(4)] 
        ## (B, Np, D) -> (B, N, D)
        ## N = (Np + 1)であることに留意
        ## また、cls_tokenの形状は(1,1,D)であるため、
        ## repeatメソッドによって(B,1,D)に変換してからパッチの埋め込みとの結合を行う 
        z_0 = torch.cat(
            [self.cls_token.repeat(repeats=(x.size(0),1,1)), z_0], dim=1)

        # 位置埋め込みの加算 [式(5)] 
        ## (B, N, D) -> (B, N, D) 
        z_0 = z_0 + self.pos_emb
        return z_0

batch_size, channel, height, width= 32, 1, 192, 192
x = torch.randn(batch_size, channel, height, width) 
input_layer = VitInputLayer(num_patch_row=2) 
z_0=input_layer(x)

# (2, 5, 384)(=(B, N, D))になっていることを確認。 
print(z_0.shape)


torch.Size([32, 5, 384])


##Multi-Head Attention

In [38]:
NUM_HEAD = 3
DROPOUT = 0.0

class MultiHeadSelfAttention(nn.Module): 
    def __init__(self, emb_dim:int=EMB_DIM, head:int=NUM_HEAD, dropout:float=DROPOUT):
        """ 
        引数:
            emb_dim: 埋め込み後のベクトルの長さ 
            head: ヘッドの数
            dropout: ドロップアウト率
        """
        super(MultiHeadSelfAttention, self).__init__() 
        self.head = head
        self.emb_dim = emb_dim
        self.head_dim = emb_dim // head
        self.sqrt_dh = self.head_dim**0.5 # D_hの二乗根。qk^Tを割るための係数

        # 入力をq,k,vに埋め込むための線形層。 [式(6)] 
        self.w_q = nn.Linear(emb_dim, emb_dim, bias=False) 
        self.w_k = nn.Linear(emb_dim, emb_dim, bias=False) 
        self.w_v = nn.Linear(emb_dim, emb_dim, bias=False)

        # 式(7)にはないが、実装ではドロップアウト層も用いる 
        self.attn_drop = nn.Dropout(dropout)

        # MHSAの結果を出力に埋め込むための線形層。[式(10)]
        ## 式(10)にはないが、実装ではドロップアウト層も用いる 
        self.w_o = nn.Sequential(
            nn.Linear(emb_dim, emb_dim),
            nn.Dropout(dropout) 
        )

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        """ 
        引数:
            z: MHSAへの入力。形状は、(B, N, D)。
                B: バッチサイズ、N:トークンの数、D:ベクトルの長さ

        返り値:
            out: MHSAの出力。形状は、(B, N, D)。[式(10)]
                B:バッチサイズ、N:トークンの数、D:埋め込みベクトルの長さ
        """

        batch_size, num_patch, _ = z.size()

        # 埋め込み [式(6)]
        ## (B, N, D) -> (B, N, D)
        q = self.w_q(z)
        k = self.w_k(z)
        v = self.w_v(z)

        # q,k,vをヘッドに分ける [式(10)]
        ## まずベクトルをヘッドの個数(h)に分ける
        ## (B, N, D) -> (B, N, h, D//h)
        q = q.view(batch_size, num_patch, self.head, self.head_dim)
        k = k.view(batch_size, num_patch, self.head, self.head_dim)
        v = v.view(batch_size, num_patch, self.head, self.head_dim)

        ## Self-Attentionができるように、
        ## (バッチサイズ、ヘッド、トークン数、パッチのベクトル)の形に変更する 
        ## (B, N, h, D//h) -> (B, h, N, D//h)
        q = q.transpose(1,2)
        k = k.transpose(1,2)
        v = v.transpose(1,2)

        # 内積 [式(7)]
        ## (B, h, N, D//h) -> (B, h, D//h, N)
        k_T = k.transpose(2, 3)
        ## (B, h, N, D//h) x (B, h, D//h, N) -> (B, h, N, N) 
        dots = (q @ k_T) / self.sqrt_dh
        ## 列方向にソフトマックス関数
        attn = F.softmax(dots, dim=-1)
        ## ドロップアウト
        attn = self.attn_drop(attn)
        # 加重和 [式(8)]
        ## (B, h, N, N) x (B, h, N, D//h) -> (B, h, N, D//h) 
        out = attn @ v
        ## (B, h, N, D//h) -> (B, N, h, D//h)
        out = out.transpose(1, 2)
        ## (B, N, h, D//h) -> (B, N, D)
        out = out.reshape(batch_size, num_patch, self.emb_dim)

        # 出力層 [式(10)]
        ## (B, N, D) -> (B, N, D) 
        out = self.w_o(out) 
        return out

mhsa = MultiHeadSelfAttention()
out = mhsa(z_0) #z_0は2-2節のz_0=input_layer(x)で、形状は(B, N, D)

# (2, 5, 384)(=(B, N, D))になっていることを確認 
print(out.shape)

torch.Size([32, 5, 384])


##Encoder

In [40]:
class VitEncoderBlock(nn.Module): 
    def __init__(self, emb_dim:int=EMB_DIM, head:int=NUM_HEAD, hidden_dim:int=EMB_DIM*4, dropout: float=DROPOUT):
        """
        引数:
            emb_dim: 埋め込み後のベクトルの長さ
            head: ヘッドの数
            hidden_dim: Encoder BlockのMLPにおける中間層のベクトルの長さ 
                        原論文に従ってemb_dimの4倍をデフォルト値としている
            dropout: ドロップアウト率
        """
        super(VitEncoderBlock, self).__init__()
        # 1つ目のLayer Normalization [2-5-2項]
        self.ln1 = nn.LayerNorm(emb_dim)
        # MHSA [2-4-7項]
        self.msa = MultiHeadSelfAttention(
        emb_dim=emb_dim, head=head,
        dropout = dropout,
        )
        # 2つ目のLayer Normalization [2-5-2項] 
        self.ln2 = nn.LayerNorm(emb_dim)
        # MLP [2-5-3項]
        self.mlp = nn.Sequential( 
            nn.Linear(emb_dim, hidden_dim), 
            nn.GELU(),
            nn.Dropout(dropout), 
            nn.Linear(hidden_dim, emb_dim), 
            nn.Dropout(dropout)
        )
    def forward(self, z: torch.Tensor) -> torch.Tensor:
        """ 
        引数:
            z: Encoder Blockへの入力。形状は、(B, N, D)
                B: バッチサイズ、N:トークンの数、D:ベクトルの長さ

        返り値:
            out: Encoder Blockへの出力。形状は、(B, N, D)。[式(10)]
                B:バッチサイズ、N:トークンの数、D:埋め込みベクトルの長さ 
        """
        # Encoder Blockの前半部分 [式(12)] 
        out = self.msa(self.ln1(z)) + z
        # Encoder Blockの後半部分 [式(13)] 
        out = self.mlp(self.ln2(out)) + out 
        return out

vit_enc = VitEncoderBlock()
z_1 = vit_enc(z_0) #z_0は2-2節のz_0=input_layer(x)で、形状は(B, N, D)

# (2, 5, 384)(=(B, N, D))になっていることを確認 
print(z_1.shape)


torch.Size([32, 5, 384])


##ViT

In [ ]:
NUM_CLASSES = 2
NUM_BLOCK = 7
HID_DIM = EMB_DIM * 4
IN_CHANNEL = 1
EMB_DIM = 384
NUM_PATCH_ROW = 3
IMG_SIZE = 64*3
NUM_HEAD = 8
DROPOUT = 0.0

class Vit(nn.Module): 
    def __init__(self, in_channels:int=IN_CHANNEL, num_classes:int=NUM_CLASSES, emb_dim:int=EMB_DIM, num_patch_row:int=NUM_PATCH_ROW, image_size:int=IMG_SIZE, num_blocks:int=NUM_BLOCK, head:int=NUM_HEAD, hidden_dim:int=HID_DIM, dropout:float=DROPOUT):
        """ 
        引数:
            in_channels: 入力画像のチャンネル数
            num_classes: 画像分類のクラス数
            emb_dim: 埋め込み後のベクトルの長さ
            num_patch_row: 1辺のパッチの数
            image_size: 入力画像の1辺の大きさ。入力画像の高さと幅は同じであると仮定 
            num_blocks: Encoder Blockの数
            head: ヘッドの数
            hidden_dim: Encoder BlockのMLPにおける中間層のベクトルの長さ 
            dropout: ドロップアウト率
        """
        super(Vit, self).__init__()
        # Input Layer [2-3節] 
        self.input_layer = VitInputLayer(
            in_channels, 
            emb_dim, 
            num_patch_row, 
            image_size)

        # Encoder。Encoder Blockの多段。[2-5節] 
        self.encoder = nn.Sequential(*[
            VitEncoderBlock(
                emb_dim=emb_dim,
                head=head,
                hidden_dim=hidden_dim,
                dropout = dropout
            )
            for _ in range(num_blocks)])

        # MLP Head [2-6-1項] 
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(emb_dim),
            nn.Linear(emb_dim, num_classes)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        引数:
            x: ViTへの入力画像。形状は、(B, C, H, W)
                B: バッチサイズ、C:チャンネル数、H:高さ、W:幅

        返り値:
            out: ViTの出力。形状は、(B, M)。[式(10)]
                B:バッチサイズ、M:クラス数 
        """
        # Input Layer [式(14)]
        ## (B, C, H, W) -> (B, N, D)
        ## N: トークン数(=パッチの数+1), D: ベクトルの長さ 
        out = self.input_layer(x)
        
        # Encoder [式(15)、式(16)]
        ## (B, N, D) -> (B, N, D)
        out = self.encoder(out)

        # クラストークンのみ抜き出す
        ## (B, N, D) -> (B, D)
        cls_token = out[:,0]

        # MLP Head [式(17)]
        ## (B, D) -> (B, M)
        pred = self.mlp_head(cls_token)
        return pred

num_classes = 10
batch_size, channel, height, width= 2, 3, 32, 32
x = torch.randn(batch_size, channel, height, width)
vit = Vit(in_channels=channel, num_classes=num_classes) 
pred = vit(x)

# (2, 10)(=(B, M))になっていることを確認 
print(pred.shape)

#Trainer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device : ",device)

device :  cuda


In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F

rng = np.random.RandomState(1234)
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



conv_net = ResNet(ResidualBlock, [3, 4, 6, 3]).to(device)
"""
conv_net = nn.Sequential(
    nn.Conv2d(3, 8, 3,padding=1),              
    nn.BatchNorm2d(8),
    nn.ReLU(),
    nn.MaxPool2d(2),                  # 16x16x32
    nn.Conv2d(8, 32, 3,padding=1),   
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),                  # 8*8*32
    nn.Conv2d(32, 128, 3,padding=1),            
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2),                  # 4x4x128
    nn.Conv2d(128, 512, 3,padding=1),            
    nn.BatchNorm2d(512),
    nn.ReLU(),
    nn.MaxPool2d(2),                  # 2x2x512
    nn.Flatten(),
    nn.Linear(2048, 256),
    nn.ReLU(),
    nn.Linear(256, 10),
)
"""


def init_weights(m):  # Heの初期化
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.0)


conv_net.apply(init_weights)


n_epochs = 15
lr = 0.01
device = 'cuda'

conv_net.to(device)
optimizer = optimizer = torch.optim.RMSprop(conv_net.parameters(), lr=lr)#torch.optim.Adam(conv_net.parameters(),lr=lr) # WRITE ME
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
# WRITE ME
loss_function = nn.CrossEntropyLoss()# WRITE ME

In [ ]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    conv_net.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
      n_train += t.size()[0]

      conv_net.zero_grad()

      x = x.to(device)
      t = t.to(device)

      y = conv_net.forward(x)

      loss  = loss_function(y,t)

      loss.backward()

      optimizer.step()

      pred = y.argmax(1)


      acc_train += (pred == t).float().sum().item()
      losses_train.append(loss.tolist())

    conv_net.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        n_val += t.size()[0]

        t = t.to(device)
        x = x.to(device)

        y = conv_net.forward(x)

        loss = loss_function(y,t)

        pred = y.argmax(1)


        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())
    scheduler.step()
    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}], learning_rate : {}'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val,
        optimizer.param_groups[0]["lr"]
    ))

EPOCH: 0, Train [Loss: 1.910, Accuracy: 0.328], Valid [Loss: 1.511, Accuracy: 0.447], learning_rate : 0.01
EPOCH: 1, Train [Loss: 1.260, Accuracy: 0.547], Valid [Loss: 1.633, Accuracy: 0.471], learning_rate : 0.01
EPOCH: 2, Train [Loss: 1.075, Accuracy: 0.618], Valid [Loss: 1.248, Accuracy: 0.595], learning_rate : 0.01
EPOCH: 3, Train [Loss: 0.955, Accuracy: 0.664], Valid [Loss: 1.159, Accuracy: 0.595], learning_rate : 0.01
EPOCH: 4, Train [Loss: 0.868, Accuracy: 0.696], Valid [Loss: 1.042, Accuracy: 0.634], learning_rate : 0.01
EPOCH: 5, Train [Loss: 0.805, Accuracy: 0.721], Valid [Loss: 1.046, Accuracy: 0.648], learning_rate : 0.01
EPOCH: 6, Train [Loss: 0.742, Accuracy: 0.740], Valid [Loss: 1.227, Accuracy: 0.635], learning_rate : 0.01
EPOCH: 7, Train [Loss: 0.695, Accuracy: 0.757], Valid [Loss: 0.933, Accuracy: 0.689], learning_rate : 0.01
EPOCH: 8, Train [Loss: 0.648, Accuracy: 0.773], Valid [Loss: 1.373, Accuracy: 0.577], learning_rate : 0.01
EPOCH: 9, Train [Loss: 0.606, Accurac

In [ ]:
conv_net.eval()

t_pred = []
for x in dataloader_test:

    x = x.to(device)

    # 順伝播
    y = conv_net.forward(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture05/submission_pred.csv', header=True, index_label='id')